i have a challenge in nlp 
Beginner’s challenge : 

Description : Email spam classification is a NLP project focused on developing a system capable of accurately distinguishing between spam and non-spam emails. The goal is to train a model using a dataset containing labeled email data, where each email is categorized as either spam or non-spam.
Dataset : they provided us with a dataset containing three files one for emails that are spam one for the ones that are not spam and the last for the ones that are not sure if spam or not

can you give me how to solve this ? i'm a beginer in nlp

In [18]:
import pandas as pd

# my data is separated (every email is in a file) and i want to make them into a csv file with the following columns
# email | label
# where label is the content of the file
# i will use pandas to do this
# the data is in the easy_ham , hard_ham and spam_2 folders

import os
import email

# get the data
data = []
for folder in ['easy_ham', 'spam_2']:
    # the emails in the easy_ham folder are not spam
    # the emails in the spam_2 folder are spam
    label = 0 if folder == 'easy_ham' else 1
    for file in os.listdir(folder):
        with open(f'{folder}/{file}', 'r', errors='replace') as f:
            
            email_str = f.read()
            data.append([email_str, label])


    

# create a dataframe
df = pd.DataFrame(data)

# save the dataframe to a csv file
df.to_csv('emails.csv', index=False)
print('done')







done


In [19]:
# read the csv file
df = pd.read_csv('emails.csv')
print(df)


                                                      0  1
0     From exmh-workers-admin@redhat.com  Thu Aug 22...  0
1     From Steve_Burt@cursor-system.com  Thu Aug 22 ...  0
2     From timc@2ubh.com  Thu Aug 22 13:52:59 2002\n...  0
3     From irregulars-admin@tb.tf  Thu Aug 22 14:23:...  0
4     From exmh-users-admin@redhat.com  Thu Aug 22 1...  0
...                                                 ... ..
3943  From tba@insiq.us  Wed Dec  4 11:46:34 2002\nR...  1
3944  Return-Path: <raye@yahoo.lv>\nReceived: from u...  1
3945  From cweqx@dialix.oz.au  Tue Aug  6 11:03:54 2...  1
3946  From ilug-admin@linux.ie  Wed Dec  4 11:52:36 ...  1
3947  mv 00001.317e78fa8ee2f54cd4890fdc09ba8176 0000...  1

[3948 rows x 2 columns]


In [34]:
"""Some hints (more like ideas to keep in mind) when working on NLP challenge 1: spam classification

As you know so far, the dataset is organized into three folders: two contain non-spam (ham) emails (text files), while the third folder contains spam emails.

All emails do follow a specific format, which will aid in parsing and extracting metadata and the mail body.

Decide on the features you want to extract from each email. Think what would help you identify if an email is a spam or not? You might need to go back to this point later to modify the model, but keep it simple to start.

 Main challenge 1: How to extract meta data and mail body from the mails text in order to generate the dataset. You can think about parsing data manually, or maybe there's a library ? 👀

Once you have managed to get the information you want from each mail, you can build the dataset, but you still need to give each one a target column (this is a supervised problem).
"""

# i will use the email module to parse the emails

# get the data
data = []

for folder in ['easy_ham', 'hard_ham', 'spam_2']:
    # the emails in the easy_ham and hard_ham folder are not spam
    # the emails in the spam_2 folder are spam
    label = 0 if folder in ['easy_ham', 'hard_ham'] else 1
    for file in os.listdir(folder):
        with open(f'{folder}/{file}', 'r', errors='replace') as f:
            email_str = f.read()
            email_obj = email.message_from_string(email_str)
            # get the body of the email
            body = ''
            if email_obj.is_multipart():
                for part in email_obj.walk():
                    content_type = part.get_content_type()
                    content_disposition = str(part.get('Content-Disposition'))
                    try:
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
            else:
                body = email_obj.get_payload(decode=True).decode()
            data.append([body, label])

# create a dataframe
df = pd.DataFrame(data, columns=['email', 'label'])

# save the dataframe to a csv file
df.to_csv('emails.csv', index=False)
print('done')

AttributeError: 'str' object has no attribute 'get_payload'

In [27]:
def remove_header(email):
    """remove the header from an email"""
    return email[email.index('\n\n'):]


In [29]:
import re

def is_url(s):
    url = re.match("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|" "[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", s)
    if url is None:
        return False
    else:
        return True
    
def url_converter(words):
    for i, word in enumerate(words):
        if is_url(word):
            words[i] = 'URL'
    return words

In [30]:
def num_converter(words):
    for i, word in enumerate(words):
        if word.isdigit():
            words[i] = 'NUM'
    return words

In [31]:
def remove_punctuation(email):
    new_email = ""
    for c in email:
        if c.isalnum() or c.isspace():
            new_email += c
    return new_email